In [ ]:

# Fix markers or kind, error bars

%matplotlib inline
# Dependencies and Setup
# get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# File to Load (Remember to Change These)
mouse_drug_data = "data/mouse_drug_data.csv"
clinical_trial_data = "data/clinicaltrial_data.csv"
drug_df = pd.read_csv(mouse_drug_data)
trial_df = pd.read_csv(clinical_trial_data)
drugtrial_df = pd.merge(drug_df, trial_df, on="Mouse ID")
drugtrial_df.head()


In [ ]:

# In[3]:
# Tumor Response to Treatment
# Store the Mean Tumor Volume Data Grouped by Drug and Timepoint
Avg_Tumor_Volume = drugtrial_df.groupby(["Drug", "Timepoint"]).agg(['mean', 'sem'])
Avg_Tumor_Volume = Avg_Tumor_Volume.reset_index()
Avg_Tumor_Volume = Avg_Tumor_Volume.drop("Metastatic Sites", axis=1)
sem = Avg_Tumor_Volume['Tumor Volume (mm3)']['sem']



In [ ]:
# In[5]:
# Minor Data Munging to Re-Format the Data Frames
Table = Avg_Tumor_Volume.pivot_table(index="Timepoint",
                                     values="Tumor Volume (mm3)",
                                     columns="Drug")
Table.head()


In [ ]:

# In[6]:
styles = ["o", ".", ",", "v", "1", "8", "s", "*", "+","|"]
fig, ax = plt.subplots()
for col, style in zip(Table.columns, styles):
    Table[col].plot(kind="line", linestyle="-", style=style, ax=ax,
                    figsize=(15, 8), xlim=(0, 48), grid=True, yerr=sem)
ax.legend(loc="Best")
ax.set_xlabel("Time (Days)")
ax.set_ylabel("Tumor Volume (mm3)")
ax.set_title("Tumor Response to Treatment")
ax.set_figsize = (15, 8)
plt.savefig('Tumor_Responses.png')

In [ ]:
# Metastatic Response to Treatment
# Store the Mean Met. Site Data Grouped by Drug and Timepoint
Metastatic_Sites = drugtrial_df.groupby(["Drug", "Timepoint"]).agg(['mean', 'sem'])
Metastatic_Sites = Metastatic_Sites.drop("Tumor Volume (mm3)", axis=1)
Metastatic_Sites = Metastatic_Sites.reset_index()
sem = Metastatic_Sites['Metastatic Sites']['sem']
Metastatic_Sites.head()

In [ ]:

# In[10]:
Table = Metastatic_Sites.pivot_table(index="Timepoint",
                                     values="Metastatic Sites",
                                     columns="Drug")
Table.head()

In [ ]:
fig, ax = plt.subplots()
for col, style in zip(Table.columns, styles):
    Table[col].plot(linestyle="-", style=style, ax=ax,
                    figsize=(15, 8), xlim=(-0.5, 48),
                    grid=True, ylim=(-0.1, 4), yerr=sem)
ax.legend(loc="Best")
ax.set_xlabel("Time (Days)")
ax.set_ylabel("Metastatic Sites")
ax.set_title("Metastatic Spread During Treatment")
ax.set_figsize = (15, 8)
plt.savefig('Metastatic_Sites.png')

In [ ]:

# In[13]:
# Survival Rates
Mouse_Count = drugtrial_df.groupby(["Drug", "Timepoint"]).count()
Mouse_Count = Mouse_Count.drop(["Metastatic Sites",
                                "Tumor Volume (mm3)"], axis=1)
Mouse_Count = Mouse_Count.reset_index()
Mouse_Count.head()


In [ ]:
# In[15]:
Mouse_pct = Mouse_Count.groupby(['Drug', 'Timepoint']).sum()
Mouse_pct = round(Mouse_pct.div(Mouse_pct.groupby('Drug').first()) * 100, 2)
Mouse_pct = Mouse_pct.rename(columns={'Mouse ID': 'Percent Alive'})

Mouse_pct.head()

In [ ]:
# In[16]:


# Minor Data Munging to Re-Format the Data Frames
Table = Mouse_pct.pivot_table(index="Timepoint",
                              values='Percent Alive', columns="Drug")
Table.head()





In [ ]:
# In[17]:
fig, ax = plt.subplots()
for col, style in zip(Table.columns, styles):
    Table[col].plot(linestyle="-", style=style, ax=ax,
                    figsize=(15, 8), xlim=(-0.5, 48), grid=True, ylim=(0, 110))
ax.legend(loc="Best")
ax.set_xlabel("Time (Days)")
ax.set_ylabel("Mouse Count")
ax.set_title("Survival During Treatment")
ax.set_figsize = (15, 8)
plt.savefig('Survival Rate.png')


In [ ]:
# Calculate the percent changes for each drug

Tumor_Change = Avg_Tumor_Volume.pivot_table(index="Timepoint",
                                            values='Tumor Volume (mm3)',
                                            columns="Drug")
Tumor_Change = round(Tumor_Change.pct_change()*100, 4)
Tumor_Change = Tumor_Change.fillna(0)
Tumor_Change = Tumor_Change.tail(1)
Tumor_Change = Tumor_Change.stack()
Tumor_Change = Tumor_Change.reset_index(level=0, drop=True)
Tumor_Change.head(15)



In [ ]:
Pct_Changed = sorted(tuple(zip(Tumor_Change['mean'], Tumor_Change.index)))
Efficacy, Drug = zip(*Pct_Changed)

colors = []
for value in Efficacy:
    if value > 0:
        colors.append('r')
    else:
        colors.append('g')

Efficacy

In [ ]:
# In[62]:
tick_locations = []
x_axis = np.arange(0, len(Drug))
for x in x_axis:
    tick_locations.append(x)
fig, ax = plt.subplots()
rects1 = ax.bar(Drug, Efficacy, color=colors, alpha=0.75, align="center")

plt.title("Drug Efficacy After 45 Days")
plt.xlabel("Drug")
plt.ylabel("% Change in Tumor Volume")
plt.xlim(-0.75, len(Drug)-.25)
plt.hlines(0, -0.75, len(Drug))
plt.xticks(tick_locations, Drug, rotation=45)
plt.savefig('Drug_Efficacy.png')


def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')

autolabel(rects1)


fig.show()